# Document Clustering and Topic Modeling 

## Contents

* [Part 1: Load Data](#Part-1:-Load-Data)
* [Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
* [Part 3: TF-IDF](#Part-3:-TF-IDF)
* [Part 4: K-means clustering](#Part-4:-K-means-clustering)
* [Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)


# Part 1: Load Data

In [ ]:
import numpy as np
import pandas as pd
import nltk
# import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load data into dataframe
df = pd.read_csv('data.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [ ]:
# Remove missing value
df.dropna(subset=['review_body'],inplace=True)

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [ ]:
# use the first 1000 data as our training data
data = df.loc[:999, 'review_body'].tolist()

In [ ]:
data

['Absolutely love this watch! Get compliments almost every time I wear it. Dainty.',
 'I love this watch it keeps time wonderfully.',
 'Scratches',
 'It works well on me. However, I found cheaper prices in other places after making the purchase',
 "Beautiful watch face.  The band looks nice all around.  The links do make that squeaky cheapo noise when you swing it back and forth on your wrist which can be embarrassing in front of watch enthusiasts.  However, to the naked eye from afar, you can't tell the links are cheap or folded because it is well polished and brushed and the folds are pretty tight for the most part.<br /><br />I love the new member of my collection and it looks great.  I've had it for about a week and so far it has kept good time despite day 1 which is typical of a new mechanical watch",
 'i love this watch for my purpose, about the people complaining should of done their research better before buying. dumb people.',
 'for my wife and she loved it, looks great and a 

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like "a", "the", or "in" which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [ ]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') #stopwords.append("n't")
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br") #html <br>
stopwords.append("watch")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 183 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [ ]:
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [ ]:
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'wear',
 'dainti']

In [ ]:
data[0]

'Absolutely love this watch! Get compliments almost every time I wear it. Dainty.'

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 1000 reviews and 239 terms.


In [ ]:
tfidf_matrix

<1000x239 sparse matrix of type '<class 'numpy.float64'>'
	with 6874 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_matrix.toarray() #todense()

array([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [ ]:
tfidf_matrix.todense()

matrix([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]])

In [ ]:
print(type(tfidf_matrix.toarray()))

<class 'numpy.ndarray'>


In [ ]:
print(type(tfidf_matrix.todense()))

<class 'numpy.matrix'>


Save the terms identified by TF-IDF.

In [ ]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [ ]:
# print out words
tf_selected_words

['abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'arm',
 'arriv',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'bracelet',
 'brand',
 'break',
 'bright',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'clasp',
 'classi',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'crown',
 'crystal',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'deliveri',
 'design',
 'dial',
 'differ',
 'difficult',
 'disappoint',
 'display',
 'dress',
 'durabl',
 'easi',
 'easili',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'fair',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'fine',
 'finish',
 'fit',
 'function',
 'gave',
 'gift',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',
 'hard',
 'heavi',
 'high',
 'hold',


# Part 4: K-means clustering

In [ ]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

## 4.1. Analyze K-means Result

In [ ]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [ ]:
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,2
1,I love this watch it keeps time wonderfully.,2
2,Scratches,4
3,"It works well on me. However, I found cheaper ...",4
4,Beautiful watch face. The band looks nice all...,4
5,"i love this watch for my purpose, about the pe...",2
6,"for my wife and she loved it, looks great and ...",0
7,I was about to buy this thinking it was a Swis...,4
8,Watch is perfect. Rugged with the metal &#34;B...,0
9,Great quality and build.<br />The motors are r...,4


In [ ]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
4,658
2,113
0,94
1,74
3,61


In [ ]:
km.cluster_centers_

# 239数的list -> cluster 0的中心点的tf-idf值
#-> assumption: 中心点的值可以代表这个cluster
#-> tf-idf值越大，对应的词越能代表这个document
#-> 选出了tf-idf最大的6个值对应的词来代表这个cluster

array([[0.00342965, 0.        , 0.        , ..., 0.0023702 , 0.        ,
        0.02477667],
       [0.        , 0.        , 0.        , ..., 0.        , 0.0088578 ,
        0.        ],
       [0.        , 0.04052046, 0.        , ..., 0.01200527, 0.01686888,
        0.0039317 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00573121, 0.00443061, 0.00388093, ..., 0.00661899, 0.01898917,
        0.01341109]])

In [ ]:
km.cluster_centers_.shape

(5, 239)

In [ ]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()

<Document clustering result by K-means>
Cluster 0 words:great,look,price,work,comfort,product,
Cluster 0 reviews (94 reviews): 
for my wife and she loved it, looks great and a great price!, Watch is perfect. Rugged with the metal &#34;Bull Bars&#34;. The red accents are a great touch and I get compliments when wearing it. If you are worried about being able to read this in sunlight or in the dark don't! The LED ilumination works great! I might even get this in a different color for my next G-Shock purchase!, Works great but the watch a used it on was slim so I had to use a quarter to rase it up the right height, Perfect Condition, Arrived on Time,Works & Looks Great, amazing product keeps everything safe and secure organized great quality for my designer belts perfect for homes or traveling, Comfortable, looks great, very lightweight.The band is a little on the short side, but it's usable and not a big deal to replace., great looking sports watch, A great automatic, but I give it 4 sta

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [ ]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [ ]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.06244924 0.05960477 0.31825296 0.05974888 0.49994414]
 [0.08454488 0.08420083 0.65506977 0.08659385 0.08959067]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10067021 0.1000004  0.10000044 0.10029237 0.59903657]
 [0.0839047  0.08319941 0.08287806 0.66495307 0.08506475]
 [0.06737232 0.06801631 0.06919343 0.0678506  0.72756735]]


In [ ]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 239)
[[0.20144327 0.20307082 0.20052626 ... 0.20026037 4.34608679 0.20233519]
 [1.87785559 0.23026411 0.20044692 ... 0.20160298 2.30885042 0.20095975]
 [2.59061652 7.6586574  0.20052757 ... 5.78185124 0.20130656 0.20055803]
 [0.21359064 0.20138702 1.19720803 ... 0.5175084  8.99883142 9.5131096 ]
 [0.21001602 0.20077291 1.75494345 ... 0.23346764 0.20146177 2.48082387]]


In [ ]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.06,0.06,0.32,0.06,0.50,4
Doc1,0.08,0.08,0.66,0.09,0.09,2
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.06,0.06,0.74,0.06,0.07,2
Doc4,0.04,0.06,0.11,0.75,0.04,3
Doc5,0.30,0.08,0.47,0.08,0.08,2
Doc6,0.06,0.74,0.06,0.07,0.06,1
Doc7,0.06,0.06,0.06,0.75,0.06,3
Doc8,0.18,0.05,0.17,0.56,0.05,3
Doc9,0.06,0.06,0.06,0.06,0.76,4


In [ ]:
df_document_topic['topic'].value_counts().to_frame()

,topic
3,294
4,242
1,175
0,154
2,135


In [ ]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[0.20144327 0.20307082 0.20052626 ... 0.20026037 4.34608679 0.20233519]
 [1.87785559 0.23026411 0.20044692 ... 0.20160298 2.30885042 0.20095975]
 [2.59061652 7.6586574  0.20052757 ... 5.78185124 0.20130656 0.20055803]
 [0.21359064 0.20138702 1.19720803 ... 0.5175084  8.99883142 9.5131096 ]
 [0.21001602 0.20077291 1.75494345 ... 0.23346764 0.20146177 2.48082387]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,anoth,arm,arriv,automat,awesom,bad,band,batteri,beauti,best,better,big,bit,black,blue,bought,box,bracelet,brand,break,bright,broke,button,buy,ca,came,case,casio,chang,cheap,...,son,star,start,stop,strap,sturdi,style,stylish,super,sure,surpris,swim,tell,thank,thing,think,thought,time,timex,tini,tri,turn,use,valu,want,watch,water,way,wear,week,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,0.201443,0.203071,0.200526,1.690965,0.203979,0.200087,0.200108,1.671027,0.920169,1.820228,0.200429,0.200892,0.204788,0.200471,3.045492,0.200560,0.200916,0.202091,2.933343,5.551675,1.629954,2.781527,0.200730,0.200787,0.200681,3.036288,1.747183,0.200247,1.966575,0.203819,1.436272,0.200049,0.200459,3.442772,0.207037,0.619561,0.200639,0.842820,0.200566,0.201144,...,5.339648,0.201159,0.200510,0.201959,0.202324,0.201198,0.460479,3.240914,5.630332,1.126945,0.200616,3.228925,0.526692,8.696606,0.200413,0.200956,0.201009,2.922457,0.200878,0.200154,0.201297,0.200184,0.201286,0.676974,2.279767,0.378330,1.771907,0.232995,0.202998,0.201244,0.201397,0.201087,0.200551,0.200365,0.207840,0.204364,0.201090,0.200260,4.346087,0.202335
Topic1,1.877856,0.230264,0.200447,2.022760,1.668108,0.200883,0.200607,0.202403,0.771354,0.240977,0.629768,0.202563,0.200998,0.203310,0.200587,0.200832,3.560657,0.200466,4.072938,0.202319,0.201228,2.073089,0.935605,1.017393,0.200649,4.627371,3.126993,0.200559,0.200395,0.202072,0.586435,0.202527,0.900188,1.226860,0.507662,4.644113,0.695520,0.200164,0.200864,5.364258,...,0.200061,0.201034,0.200325,1.247418,0.200847,0.200147,0.204595,0.200981,0.201246,2.664906,0.207380,0.503798,1.710189,0.203629,0.606170,0.201606,0.820785,2.194649,0.200247,2.481964,5.438706,0.200049,1.332824,0.201278,0.202313,0.201090,0.201012,0.202670,2.652472,0.360626,0.200795,0.201170,11.203954,1.760957,0.205911,3.653594,0.525666,0.201603,2.308850,0.200960
Topic2,2.590617,7.658657,0.200528,0.202256,0.658204,0.201112,0.200698,0.201075,1.394976,0.926695,0.200403,2.048047,0.586817,2.311805,0.201734,0.201257,6.026709,1.219653,22.924781,0.203952,4.594115,0.208016,0.207056,1.249216,0.203139,5.133101,0.727330,1.647644,1.788406,0.290745,0.215151,0.200485,0.200173,0.660598,0.201885,1.257900,1.096231,0.495988,0.200347,1.149185,...,0.201396,0.773415,0.200847,0.201044,0.201165,0.201109,0.952900,0.201213,0.201264,1.290991,0.201825,0.200969,0.201097,0.201731,0.202981,0.202223,4.108620,3.377714,0.200349,1.476832,0.967850,0.653041,0.221866,0.202603,0.894029,0.964285,0.202144,3.934585,3.687524,0.200826,0.200842,0.201393,0.201482,1.861374,3.426225,1.388720,0.203165,5.781851,0.201307,0.200558
Topic3,0.213591,0.201387,1.197208,1.735809,3.554154,4.445950,0.204071,0.203576,7.478485,2.264158,1.677040,0.205479,3.362821,2.233207,9.972686,3.976071,24.721930,4.584559,0.205012,0.207378,3.414728,3.282103,6.611608,4.653933,5.969634,0.206987,0.203780,2.555728,0.212612,0.205915,0.902766,0.202586,3.567284,6.198147,2.607069,2.157616,1.395167,3.817461,5.497755,4.106713,...,0.203338,4.058260,0.208202,0.203853,13.323453,2.472461,0.205517,1.865341,0.202716,0.269829,3.969668,1.376938,5.178546,0.205278,5.816998,5.032117,0.207095,13.292065,4.770108,0.212365,1.404537,4.636699,9.954895,0.984450,0.331327,3.459500,1.647997,1.414546,5.402786,3.390450,4.915675,1.432185,0.200834,0.208787,0.227504,7.634569,3.070335,0.517508,8.998831,9.513110
Topic4,0.210016,0.200773,1.754943,0.201670,0.205031,0.891082,4.956510,2.560852,0.200583,1.590007,5.142811,2.107704,3.891998,0.201735,0.201184,5.856506,2.644397,6.992052,0.201164,0.202762,0.204173,9.307827,0.201979,0.209477,1.223208,2.433059,0.201402,0.200785,0.970418,3.847737,0.551274,10.114863,0.202079,4.807667,2.978494,1.033077,3.336465,0.200936,0.341557,1.017235,...,0.201570,1.681124,4.395116,7.027559,0.201498,2.034732,7.992491,2.728938,1.766734,1.379149,0.204475,0.204843,0.201161,0.201821,0.203795,4.110948,0.201763,15.061875,0.202137,0.202566,0.203928,0.200133,8.571029,5.784773,11.510926,12.654685

In [ ]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,perfect,excel,product,great,fast,thank,seller,pleas,deliveri,happi,good,cool,compliment,ship,qualiti
Topic 1,nice,look,love,wife,price,gift,great,heavi,got,simpl,realli,movement,tri,pictur,cheap
Topic 2,love,beauti,expect,pretti,return,disappoint,absolut,color,band,clock,face,worth,bought,purchas,order
Topic 3,great,band,look,light,strap,time,hand,awesom,use,year,replac,wrist,color,design,like
Topic 4,good,work,like,time,look,wear,watch,want,day,great,broke,qualiti,big,use,week
